In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import pickle

In [3]:
df = pd.read_csv(r"C:\Users\PORTABLE\Desktop\projet_annuel\core\plank_model\data\plank_dataset_keypoints.csv")
print(df.shape)
df.head()

(3935, 69)


,label,nose_x,nose_y,nose_z,nose_v,left_shoulder_x,left_shoulder_y,left_shoulder_z,left_shoulder_v,right_shoulder_x,...,right_heel_z,right_heel_v,left_foot_index_x,left_foot_index_y,left_foot_index_z,left_foot_index_v,right_foot_index_x,right_foot_index_y,right_foot_index_z,right_foot_index_v
0,0,0.830130,0.351851,-0.025262,0.999119,0.716336,0.299204,0.223861,0.993920,0.721056,...,-0.051480,0.831482,0.031709,0.376000,0.180639,0.640694,0.002836,0.391812,-0.140807,0.875498
1,0,0.827971,0.351691,-0.026928,0.999033,0.716258,0.299152,0.236521,0.993811,0.721311,...,-0.075619,0.827558,0.033213,0.375629,0.180146,0.626701,0.011874,0.391841,-0.171767,0.869638
2,0,0.826663,0.350897,-0.034896,0.998833,0.717475,0.299097,0.238402,0.993522,0.722325,...,-0.074106,0.822614,0.033985,0.374774,0.204634,0.610258,0.014278,0.392639,-0.170130,0.862897
3,0,0.825896,0.350415,-0.040929,0.998722,0.717678,0.299094,0.238142,0.993332,0.722549,...,-0.060950,0.820857,0.037479,0.374753,0.218800,0.599725,0.018048,0.393976,-0.154337,0.859343
4,0,0.825994,0.350126,-0.047208,0.998716,0.717702,0.299497,0.231987,0.993373,0.722911,...,-0.061091,0.820887,0.038908,0.374778,0.217422,0.595844,0.021588,0.394186,-0.154317,0.858819


In [4]:
# Load ALREADY-SPLIT datasets (SOURCE OF TRUTH)
train_df = pd.read_csv(r"C:\Users\PORTABLE\Desktop\projet_annuel\core/plank_model/data/plank_train_keypoints.csv")
test_df  = pd.read_csv(r"C:\Users\PORTABLE\Desktop\projet_annuel\core/plank_model/data/plank_test_keypoints.csv")

X_train = train_df.drop("label", axis=1).values.astype(np.float32)
y_train = train_df["label"].values.astype(np.float32)

X_test = test_df.drop("label", axis=1).values.astype(np.float32)
y_test = test_df["label"].values.astype(np.float32)

# Fit scaler ONLY on TRAIN
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)

# Save scaler for live
with open(r"C:\Users\PORTABLE\Desktop\projet_annuel\core/plank_model/model/scaler_keypoints.pkl", "wb") as f:
    pickle.dump(scaler, f)

# Torch tensors
X_train = torch.tensor(X_train)
y_train = torch.tensor(y_train)

X_test = torch.tensor(X_test)
y_test = torch.tensor(y_test)

train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=32, shuffle=True)
test_loader  = DataLoader(TensorDataset(X_test, y_test), batch_size=32)

# SANITY CHECK
print("TRAIN:", X_train.shape, "TEST:", X_test.shape)
print("Label distrib train:", np.unique(y_train.numpy(), return_counts=True))
print("Label distrib test :", np.unique(y_test.numpy(), return_counts=True))

TRAIN: torch.Size([3148, 68]) TEST: torch.Size([787, 68])
Label distrib train: (array([0., 1.], dtype=float32), array([2145, 1003]))
Label distrib test : (array([0., 1.], dtype=float32), array([536, 251]))


In [5]:
class MLP_Keypoints(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(64, 1)
        )

    def forward(self, x):
        return self.net(x)

model = MLP_Keypoints(input_dim=X_train.shape[1])

In [6]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [7]:
EPOCHS = 20

for epoch in range(EPOCHS):

    model.train()
    total_loss = 0

    for xb, yb in train_loader:
        preds = model(xb).squeeze()
        loss = loss_fn(preds, yb)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{EPOCHS} - Loss: {total_loss/len(train_loader):.4f}")


Epoch 1/20 - Loss: 0.2574
Epoch 2/20 - Loss: 0.0712
Epoch 3/20 - Loss: 0.0339
Epoch 4/20 - Loss: 0.0200
Epoch 5/20 - Loss: 0.0139
Epoch 6/20 - Loss: 0.0097
Epoch 7/20 - Loss: 0.0093
Epoch 8/20 - Loss: 0.0094
Epoch 9/20 - Loss: 0.0041
Epoch 10/20 - Loss: 0.0057
Epoch 11/20 - Loss: 0.0031
Epoch 12/20 - Loss: 0.0032
Epoch 13/20 - Loss: 0.0055
Epoch 14/20 - Loss: 0.0032
Epoch 15/20 - Loss: 0.0038
Epoch 16/20 - Loss: 0.0009
Epoch 17/20 - Loss: 0.0014
Epoch 18/20 - Loss: 0.0018
Epoch 19/20 - Loss: 0.0107
Epoch 20/20 - Loss: 0.0008


In [8]:
model.eval()
with torch.no_grad():
    preds = model(X_test).squeeze()
    preds = torch.sigmoid(preds)
    preds = (preds > 0.5).float()

accuracy = (preds == y_test).float().mean()
print("Accuracy:", accuracy.item())

Accuracy: 1.0


In [9]:
EPOCHS = 15
train_losses = []
test_losses = []
best_loss = float("inf")
patience = 2
wait = 0

for epoch in range(EPOCHS):

    model.train()
    total_loss = 0

    for xb, yb in train_loader:
        preds = model(xb).squeeze()
        loss = loss_fn(preds, yb)

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        total_loss += loss.item()

    train_losses.append(total_loss / len(train_loader))

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for xb, yb in test_loader:
            preds = model(xb).squeeze()
            loss = loss_fn(preds, yb)
            val_loss += loss.item()

    val_loss /= len(test_loader)
    test_losses.append(val_loss)

    print(f"Epoch {epoch+1}/{EPOCHS} - Train Loss: {train_losses[-1]:.4f} - Test Loss: {val_loss:.4f}")

Epoch 1/15 - Train Loss: 0.0005 - Test Loss: 0.0000
Epoch 2/15 - Train Loss: 0.0007 - Test Loss: 0.0000
Epoch 3/15 - Train Loss: 0.0004 - Test Loss: 0.0000
Epoch 4/15 - Train Loss: 0.0005 - Test Loss: 0.0001
Epoch 5/15 - Train Loss: 0.0010 - Test Loss: 0.0001
Epoch 6/15 - Train Loss: 0.0004 - Test Loss: 0.0001
Epoch 7/15 - Train Loss: 0.0005 - Test Loss: 0.0030
Epoch 8/15 - Train Loss: 0.0011 - Test Loss: 0.0013
Epoch 9/15 - Train Loss: 0.0015 - Test Loss: 0.0002
Epoch 10/15 - Train Loss: 0.0005 - Test Loss: 0.0000
Epoch 11/15 - Train Loss: 0.0002 - Test Loss: 0.0000
Epoch 12/15 - Train Loss: 0.0001 - Test Loss: 0.0000
Epoch 13/15 - Train Loss: 0.0001 - Test Loss: 0.0000
Epoch 14/15 - Train Loss: 0.0002 - Test Loss: 0.0000
Epoch 15/15 - Train Loss: 0.0022 - Test Loss: 0.0001


In [11]:
torch.save(model.state_dict(), r"C:\Users\PORTABLE\Desktop\projet_annuel\core\plank_model\model\plank_mlp_keypoints.pt")
print("Modèle KEYPOINTS sauvegardé.")

Modèle KEYPOINTS sauvegardé.
